# Experiment:

Explore the dataset and build a model that can accept the data

In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [36]:
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image
#from torchnet.meter import AverageValueMeter
import torch.backends.cudnn as cudnn

In [38]:
from utils import *
from model import *

In [104]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')
# Change the current working directory
os.chdir('/content/gdrive/MyDrive/')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [106]:
%pwd

'/content/gdrive/MyDrive'

In [115]:
import argparse
import torch

# Default parameters
default_params = {
    'data_dir' : '/content/gdrive/MyDrive/data/',
    'nb_epoch': 50,
    'test_size': 0.1,
    'learning_rate': 0.0001,
    'samples_per_epoch': 64,
    'batch_size': 36,
    'cuda': True,
    'seed': 7
}

# Parse command-line arguments
args = argparse.Namespace(**default_params)

# Adjust CUDA setting based on availability
args.cuda = args.cuda and torch.cuda.is_available()

# Set random seed for reproducibility
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [116]:
columns = ["center", "left", "right", "steering", "throttle", "reverse", "speed"]
df = pd.read_csv(os.path.join(args.data_dir,"driving_log.csv"))

X = df[['center', 'left', 'right']].values
y = df['steering'].values

# Split the data into training (80%), testing (20%), and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=args.test_size, random_state=0, shuffle=True)

In [117]:
df.tail()

,center,left,right,steering,throttle,brake,speed
8031,IMG/center_2016_12_01_13_46_38_745.jpg,IMG/left_2016_12_01_13_46_38_745.jpg,IMG/right_2016_12_01_13_46_38_745.jpg,0.0,0.0,0.0,1.402436
8032,IMG/center_2016_12_01_13_46_38_802.jpg,IMG/left_2016_12_01_13_46_38_802.jpg,IMG/right_2016_12_01_13_46_38_802.jpg,0.0,0.0,0.0,1.393976
8033,IMG/center_2016_12_01_13_46_38_846.jpg,IMG/left_2016_12_01_13_46_38_846.jpg,IMG/right_2016_12_01_13_46_38_846.jpg,0.0,0.0,0.0,1.388364
8034,IMG/center_2016_12_01_13_46_38_922.jpg,IMG/left_2016_12_01_13_46_38_922.jpg,IMG/right_2016_12_01_13_46_38_922.jpg,0.0,0.0,0.0,1.377208
8035,IMG/center_2016_12_01_13_46_38_947.jpg,IMG/left_2016_12_01_13_46_38_947.jpg,IMG/right_2016_12_01_13_46_38_947.jpg,0.0,0.0,0.0,1.374433


In [110]:
transformations = transforms.Compose([transforms.Lambda(lambda x: x/127.5 - 1)
                                     ])

In [118]:
#train_set = CarDataset(X_train, y_train, args.data_dir, False,transformations)
train_set = CarDataset3Img(X_train, y_train, args.data_dir,transformations)

In [119]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)

# Iterate through the DataLoader to get a batch
for batch in train_loader:
  images, labels = batch['images'], batch['labels']
  # Process the batch as needed
  print(f"Batch shape - Images: {images.shape}, Labels: {labels.shape}")
  # You can now use the batch for training or other purposes
  break  # Stop after the first batch for demonstration

FileNotFoundError: ignored

In [98]:
os.getcwd()

'/content/gdrive/MyDrive'

In [62]:
valid_set = CarDataset(X_valid, y_valid, args.data_dir, False, transformations)

In [63]:
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [64]:
valid_loader = DataLoader(valid_set, batch_size=args.batch_size, shuffle=False, num_workers=4)

# Build Model

 ### NVIDIA Model
    Image normalization to avoid saturation and make gradients work better.
    Convolution: 5x5, filter: 24, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 36, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 48, strides: 2x2, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Drop out (0.5)
    Fully connected: neurons: 100, activation: ELU
    Fully connected: neurons: 50, activation: ELU
    Fully connected: neurons: 10, activation: ELU
    Fully connected: neurons: 1 (output)
    
    the convolution layers are meant to handle feature engineering
    the fully connected layer for predicting the steering angle.
    dropout avoids overfitting
    ELU(Exponential linear unit) function takes care of the Vanishing gradient problem.
    
### Simple Model

```
  (module): CarSimpleModel (
    (conv_layers): Sequential (
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (1): ELU (alpha=1.0)
      (2): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (3): MaxPool2d (size=(4, 4), stride=(4, 4), dilation=(1, 1))
      (4): Dropout (p = 0.25)
    )
    (linear_layers): Sequential (
      (0): Linear (3648 -> 50)
      (1): ELU (alpha=1.0)
      (2): Linear (50 -> 10)
      (3): Linear (10 -> 1)
    )
  )
```

# Define Train

In [65]:
def toVariable(data, use_cuda):
    input, target = data
    input, target = Variable(input.float()), Variable(target.float())
    if use_cuda:
        input, target = input.cuda(), target.cuda()

    return input, target

In [66]:
# Training
def train(epoch, net, dataloader, optimizer, criterion, use_cuda):
    net.train()
    train_loss = 0

    for batch_idx, (centers, lefts, rights) in enumerate(dataloader):

        optimizer.zero_grad()
        centers, lefts, rights = toVariable(centers, use_cuda), \
                                 toVariable(lefts, use_cuda), \
                                 toVariable(rights, use_cuda)
        datas = [lefts, rights, centers]
        for data in datas:
            imgs, targets = data
            outputs = net(imgs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.data[0]

        if batch_idx % 100 == 0:
            print('Loss: %.3f '
                % (train_loss/((batch_idx+1)*3)))

## Define Test

In [67]:
def valid(epoch, net, validloader, criterion, use_cuda):
    global best_loss
    net.eval()
    valid_loss = 0
    for batch_idx, (inputs, targets) in enumerate(validloader):
        inputs, targets = Variable(inputs.float()), Variable(targets.float())
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        outputs = net(inputs)
        loss = criterion(outputs, targets)

        valid_loss += loss.data[0]

        avg_valid_loss = valid_loss/(batch_idx+1)
        if batch_idx % 100 == 0:
            print('Valid Loss: %.3f '
                % (valid_loss/(batch_idx+1)))
        if avg_valid_loss <= best_loss:
            best_loss = avg_valid_loss
            print('Best epoch: ' + str(epoch))
            state = {
                'net': net.module if args.cuda else net,
            }
            torch.save(state, './model3.h5')

In [68]:
net = CarSimpleModel()
optimizer = optim.Adam(net.parameters(), lr=args.learning_rate)

if args.cuda:
    net.cuda()
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True

criterion = nn.MSELoss()

/content/model.py:153: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight, mean=0, std=0.02)


In [69]:
best_loss = 0.999

In [70]:
for epoch in range(0,15):
    #optimizer = lr_scheduler(optimizer, epoch, lr_decay_epoch=args.lr_decay_epoch)
    print('\nEpoch: %d' % epoch)
    train(epoch, net, train_loader, optimizer, criterion, args.cuda)
    valid(epoch, net, valid_loader, criterion, args.cuda)


Epoch: 0


FileNotFoundError: ignored

## Save Model

In [ ]:
state = {
        'net': net.module if args.cuda else net,
        }

In [ ]:
torch.save(state, './model4.h5')